## Проект по SQL

### Описание проекта
Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. В нашем распоряжении имеется база данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

Цель проекта: проанализировать базу данных.

### Описание данных

Таблица books содержит данные о книгах:
- book_id — идентификатор книги
- author_id — идентификатор автора
- title — название книги
- num_pages — количество страниц
- publication_date — дата публикации книги
- publisher_id — идентификатор издателя

Таблица authors содержит данные об авторах:
- author_id — идентификатор автора
- author — имя автора

Таблица publishers содержит данные об издательствах:
- publisher_id — идентификатор издательства
- publisher — название издательства

Таблица ratings содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки
- book_id — идентификатор книги
- username — имя пользователя, оставившего оценку
- rating — оценка книги

Таблица reviews содержит данные о пользовательских обзорах:
- review_id — идентификатор обзора
- book_id — идентификатор книги
- username — имя автора обзора
- text — текст обзора

### План выполнения задания
- Описание исследования
- Исследование таблиц — вывод первых строк
- Сделать по одному SQL-запросу для решения каждого задания
- Вывести результаты каждого запроса в тетрадке
- Описать выводы по каждой из решённых задач

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [ ]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 1'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

## Исследование таблиц

### Таблица books содержит данные о книгах

In [ ]:
query = '''
SELECT COUNT(*)
FROM books
LIMIT 5
'''
pd.io.sql.read_sql(sql=text(query), con = con)

In [ ]:
query = '''
SELECT *
FROM books
LIMIT 5
'''
pd.io.sql.read_sql(sql=text(query), con = con)

### Таблица authors содержит данные об авторах:

In [ ]:
query = '''
SELECT COUNT(*)
FROM authors
'''
pd.io.sql.read_sql(sql=text(query), con = con)

In [ ]:
query = '''
SELECT *
FROM authors
LIMIT 5
'''
pd.io.sql.read_sql(sql=text(query), con = con)

### Таблица publishers содержит данные об издательствах:

In [ ]:
query = '''
SELECT COUNT(*)
FROM publishers
'''
pd.io.sql.read_sql(sql=text(query), con = con)

In [ ]:
query = '''
SELECT *
FROM publishers
LIMIT 5
'''
pd.io.sql.read_sql(sql=text(query), con = con)

### Таблица ratings содержит данные о пользовательских оценках книг:


In [ ]:
query = '''
SELECT COUNT(*)
FROM ratings
'''
pd.io.sql.read_sql(sql=text(query), con = con)

In [ ]:
query = '''
SELECT *
FROM ratings
LIMIT 5
'''
pd.io.sql.read_sql(sql=text(query), con = con)

### Таблица reviews содержит данные о пользовательских обзорах:


In [ ]:
query = '''
SELECT COUNT(*)
FROM reviews
'''
pd.io.sql.read_sql(sql=text(query), con = con)

In [ ]:
query = '''
SELECT *
FROM reviews
LIMIT 5
'''
pd.io.sql.read_sql(sql=text(query), con = con)

### Вывод
- В таблице books содержится информация о 1000 книгах
- В таблице authors содержится информация о 636 авторах
- В таблице publishers содержится информация о 340 издательствах
- В таблице ratings содержится информация о 6456 оценках
- В таблице reviews содержится информация о 2793 пользовательских обзорах

## SQL-запросы для решения каждого задания

### Посчитаю, сколько книг вышло после 1 января 2000 года

In [ ]:
query = '''
SELECT COUNT(DISTINCT book_id) as kolichestvo_knig
FROM books
WHERE publication_date > '2000-01-01'
'''
pd.io.sql.read_sql(sql=text(query), con = con)

### Вывод
- После 1 января вышло 819 книг

### Для каждой книги посчитаю количество обзоров и среднюю оценку

In [ ]:
query = '''
SELECT title as nazvanie,
       COUNT(DISTINCT review_id) as kolichestvo_obzorov,
       AVG(rating) as sredni_rating
FROM books as b
LEFT JOIN reviews as r ON b.book_id = r.book_id
FULL JOIN ratings as re ON b.book_id = re.book_id
GROUP BY nazvanie
ORDER BY kolichestvo_obzorov DESC
'''
pd.io.sql.read_sql(sql=text(query), con = con)

### Вывод
- Максимальное количество обзоров (8) и рейтинг (4.138) имеет книга Мемуары Гейши

### Определю издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим  из анализа брошюры

In [ ]:
query = '''
SELECT publisher as izdatelstvo,
       COUNT(book_id) as kolichestvo_knig
FROM books as b
INNER JOIN publishers as pbs ON b.publisher_id = pbs.publisher_id
WHERE num_pages > 50
GROUP BY izdatelstvo
ORDER BY kolichestvo_knig DESC
LIMIT 1
'''
pd.io.sql.read_sql(sql=text(query), con = con)

### Вывод 
- Издательство, которое выпустило наибольшее число книг(42) толще 50 страниц именуется Penguin Books

### Определю автора с самой высокой средней оценкой книг — учитываю только книги с 50 и более оценками

ЭТО ДЕЛАЛ ДЛЯ СЕБЯ, ЧТОБЫ ТАК СКАЗАТЬ, НЕ ДЕРЖАТЬ ЧАСТЬ КОДА В ГОЛОВЕ:
Тут решил сделать декомпозицию и разбить задачу на несколько частей. Сначала вывел таблицу с айди книги, авторами и названиями книг

In [ ]:
query = '''
SELECT author,
       book_id,
       title           
FROM books as b
LEFT JOIN authors as atr ON b.author_id = atr.author_id
'''
pd.io.sql.read_sql(sql=text(query), con = con)

Затем вывел таблицу с айди книги, количеством оценок и средним рейтингом

In [ ]:
query = '''
SELECT DISTINCT b.book_id,
       COUNT(rating) as kolichestvo_ocenok,
       AVG(rating) as srednaya_ocenka
FROM books as b
LEFT JOIN ratings as rtg ON b.book_id = rtg.book_id
GROUP BY b.book_id
ORDER BY b.book_id ASC

'''
pd.io.sql.read_sql(sql=text(query), con = con)

С помощью табличного выражения получил нужный результат

In [ ]:
query = '''
WITH
x AS (SELECT book_id,
             title,
             author       
      FROM books as b
      LEFT JOIN authors as atr ON b.author_id = atr.author_id),

y AS (SELECT DISTINCT b.book_id,
             COUNT(rating) as kolichestvo_ocenok,
             AVG(rating) as srednaya_ocenka
      FROM books as b
      LEFT JOIN ratings as rtg ON b.book_id = rtg.book_id
      GROUP BY b.book_id
      ORDER BY COUNT(rating) DESC)

SELECT x.author,
       AVG(y.srednaya_ocenka) AS srednaya_ocenka
FROM x 
LEFT OUTER JOIN y ON x.book_id = y.book_id
WHERE kolichestvo_ocenok > 50
GROUP BY x.author
ORDER BY srednaya_ocenka DESC
LIMIT 1
'''
pd.io.sql.read_sql(sql=text(query), con = con)

### Вывод 
Автор с самой высокой средней оценкой книг J.K. Rowling/Mary GrandPré, а средняя оценка 4.283844

### Посчитаю среднее количество обзоров от пользователей, которые поставили больше 48 оценок

ЭТО ДЕЛАЛ ДЛЯ СЕБЯ, ЧТОБЫ ТАК СКАЗАТЬ, НЕ ДЕРЖАТЬ ЧАСТЬ КОДА В ГОЛОВЕ:
Так же использовал декомпозицию. Сначала нашёл пользователей и количество обзоров

In [ ]:
query = '''
SELECT DISTINCT username,
       COUNT(text) as kolichestvo_obzorov
FROM books as b
LEFT JOIN reviews as rvw ON b.book_id = rvw.book_id
GROUP BY username
ORDER BY kolichestvo_obzorov DESC
'''
pd.io.sql.read_sql(sql=text(query), con = con)

Затем нашёл пользователей и количество оценок

In [ ]:
query = '''
SELECT username,
       COUNT(rating) as kolichestvo_ocenok
FROM books as b
LEFT JOIN ratings as rtg ON b.book_id = rtg.book_id
GROUP BY username
ORDER BY COUNT(rating) DESC

'''
pd.io.sql.read_sql(sql=text(query), con = con)

С помощью табличного выражения получил нужный результат

In [ ]:
query = '''
WITH
x AS (SELECT DISTINCT username,
             COUNT(text) as kolichestvo_obzorov
      FROM books as b
      LEFT JOIN reviews as rvw ON b.book_id = rvw.book_id
      GROUP BY username
      ORDER BY kolichestvo_obzorov DESC),

y AS (SELECT username,
             COUNT(rating) as kolichestvo_ocenok
      FROM books as b
      LEFT JOIN ratings as rtg ON b.book_id = rtg.book_id
      GROUP BY username
      ORDER BY COUNT(rating) DESC)

SELECT AVG(x.kolichestvo_obzorov) as srednee
FROM x 
LEFT OUTER JOIN y ON x.username = y.username
WHERE kolichestvo_ocenok > 48
'''
pd.io.sql.read_sql(sql=text(query), con = con)

### Вывод 
- Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок около 24